In [2]:
from scienceworld import ScienceWorldEnv
from sources.agent import BDIAgent
from sources.scienceworld import parse_observation

from sources.bdi_components.belief import State
from sources.bdi_components.inference import NLIModel
from sources.bdi_components.plans import PlanLibrary

In [3]:
hg_model_hub_name = "alisawuffles/roberta-large-wanli"
nli_model = NLIModel(hg_model_hub_name)

In [4]:


pl = PlanLibrary()
pl.load_plans_from_file("../plans/plans_nl/plan_1_melt.plan")  # load plans from strings above
pl.load_plans_from_file("plans_navigation.txt")  # load plans from file
print(pl.plans.keys())

dict_keys(['melt caesium', 'get metal pot', 'get caesium', 'heat caesium on blast furnace', 'move to art studio', 'move to bedroom', 'move to greenhouse', 'move to kitchen', 'move to living room', 'move to workshop', 'move to outside', 'move to foundry', 'move to bathroom', 'move to hallway'])


In [5]:
env = ScienceWorldEnv("", "", envStepLimit=100)

#root_event = 'use chemistry to create green paint'

task = 'melt'
env.load(task, 0)
#randVariationIdx = env.getRandomVariationTest()
randVariationIdx = 28
env.load(task, randVariationIdx)

goal = env.getTaskDescription().split('.')[0].replace("Your task is to", "").strip()

print(f"Task Name: " + 'boil' + " variation " + str(randVariationIdx))
print("Task Description: " + str(env.getTaskDescription()))

# Reset the environment
observation, info = env.reset()
# initial state
observation, reward, isCompleted, info = env.step('look around')
current_state = parse_observation(observation=observation, inventory=info['inv'], look_around=info['look'],
                                  task=goal, valid_actions=info['valid'])

goal

Task Name: boil variation 28
Task Description: Your task is to melt caesium. First, focus on the substance. Then, take actions that will cause it to change its state of matter.


'melt caesium'

In [6]:
def step_function(action: str) -> State:
    observation, reward, isCompleted, info = env.step(action)
    updated_state = parse_observation(observation=observation,
                                      inventory=info['inv'],
                                      look_around=info['look'],
                                      task=goal,
                                      valid_actions=info['valid'],
                                      score=info['score'])
    print(f"Action: {action} -> Obs {observation}")
    return updated_state


agent = BDIAgent(plan_library=pl, nli_model=nli_model)
last_state = agent.act(current_state, step_function=step_function)

print(env.getGoalProgressStr())
print(last_state.reward)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


event -> melt caesium -> state False
event -> move to kitchen -> state False
Action: open door to kitchen -> Obs The door is now open.
Action: go to kitchen -> Obs You move to the kitchen.
Action: pick up thermometer -> Obs You move the thermometer to the inventory.
event -> get metal pot -> state False
Action: open cupboard -> Obs The cupboard is now open.
Action: pick up metal pot -> Obs You move the metal pot to the inventory.
event -> get caesium -> state False
event -> move to workshop -> state False
Action: open door to hallway -> Obs The door is now open.
Action: go to hallway -> Obs You move to the hallway.
Action: open door to workshop -> Obs The door is now open.
Action: go to workshop -> Obs You move to the workshop.
Action: pick up caesium -> Obs You move the caesium to the inventory.
Action: focus on caesium -> Obs You focus on the caesium.
Action: move caesium to metal pot -> Obs You move the caesium to the metal pot.
event -> heat caesium on blast furnace -> state False
